In [ ]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass
!pip install triton transformers
!pip install -U datasets
!pip install --pre -U xformers ##### this take some time

### ****Note**: Restart the Kernel after package installation**

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_token")

### 1. Load the model

In [2]:
from unsloth import FastLanguageModel
import torch
from IPython.display import display_markdown
max_seq_length = 512 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/llama-3-8b-bnb-4bit",  
]  #### loadin llama 3 model in 4 bit to fine tune

/opt/conda/lib/python3.10/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-04-22 09:59:01.153167: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-22 09:59:01.153283: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-22 09:59:01.300116: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be somethin

Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


/opt/conda/lib/python3.10/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [17]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30.dev1005. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [18]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

### 2. Load the dataset

In [5]:
!pip install -q -U sentencepiece
!pip install -q -U datasets
!pip install -q -U sacrebleu

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, concatenate_datasets

import sacrebleu
RANDOM_SEED = 42

with open('/kaggle/input/train-eng-bdq-phase2dataset-v1/bdq-eng.train.bdq', 'r', encoding='utf-8') as file_ba:
    ba_data = file_ba.readlines()
with open('/kaggle/input/train-eng-bdq-phase2dataset-v1/bdq-eng.train.eng', 'r', encoding='utf-8') as file_en:
    en_data = file_en.readlines()
assert len(ba_data) == len(en_data), "The files don't have the same number of lines."
train_df = pd.DataFrame({'English': en_data, 'Bahnar': ba_data})
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=RANDOM_SEED)

ba_en_train_dataset = Dataset.from_pandas(train_df)
ba_en_val_dataset = Dataset.from_pandas(val_df)

with open('/kaggle/input/test-eng-bdq-phase2dataset-v1/bdq-eng.test.bdq', 'r', encoding='utf-8') as file_ba:
    ba_data = file_ba.readlines()
with open('/kaggle/input/test-eng-bdq-phase2dataset-v1/bdq-eng.test.eng', 'r', encoding='utf-8') as file_en:
    en_data = file_en.readlines()
assert len(ba_data) == len(en_data), "The files don't have the same number of lines."
test_df = pd.DataFrame({'English': en_data, 'Bahnar': ba_data})
ba_en_test_dataset = Dataset.from_pandas(test_df)

print(len(ba_en_train_dataset), len(ba_en_val_dataset), len(ba_en_test_dataset))

21801 2423 1000


#### Map each sample to the prompt template

In [7]:
# Must add EOS_TOKEN at response last line
EOS_TOKEN = tokenizer.eos_token 

# EOS_TOKEN = "PIKACHU"
def mapping_response(sample):
    sample['text'] = "You are an expert Bahnar translator! Below is an instruction that describes a task. Write a response that appropriately completes the request.  ### Instruction: \n Translate the Bahnar input text into English. \n"+sample['Bahnar']+"\n ### Response: \n"+sample['English']+EOS_TOKEN
    return sample

In [8]:
ba_en_train_dataset = ba_en_train_dataset.map(mapping_response)
ba_en_val_dataset = ba_en_val_dataset.map(mapping_response)

Map:   0%|          | 0/21801 [00:00<?, ? examples/s]

Map:   0%|          | 0/2423 [00:00<?, ? examples/s]

In [9]:
ba_en_test_dataset = ba_en_test_dataset.map(mapping_response)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Optional: Select 32 good entries from the test dataset for fine tuning

In [10]:
good_entries = [5, 99, 96, 89, 87, 84, 83, 77, 64, 65, 66, 61, 58, 46, 42, 49, 48, 47, 26, 23, 100, 107, 109, 112, 141, 175, 176, 177, 196, 198, 171, 157]
good_32 = ba_en_test_dataset.select(good_entries)

In [11]:
def prompt_inference(prmpt):
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        prmpt
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 512, temperature=0.1, use_cache = True)
    return tokenizer.batch_decode(outputs)[0].split("### Response:")[-1].split(EOS_TOKEN)[0]

In [12]:
prompt = "You are an expert Bahnar translator! Below is an instruction that describes a task. Write a response that appropriately completes the request.  ### Instruction: \n Translate the Bahnar input text into English. \n" + ba_en_test_dataset['Bahnar'][90] +"\n ### Response: \n"
print(prompt)

You are an expert Bahnar translator! Below is an instruction that describes a task. Write a response that appropriately completes the request.  ### Instruction: 
 Translate the Bahnar input text into English. 
Thoi noh kăl khŏm mă ming hơmet rai hrôih rai 'lơ̆ng.

 ### Response: 



In [13]:
print("result")
display_markdown(prompt_inference(prmpt=prompt),raw=True)

result


 
 The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated into English as follows: 
The Bahnar input text is translated

### 3. Training

#### Setup the training args

In [19]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = good_32,
    dataset_text_field = "text", ### taking text column from the dataset
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        report_to=None,
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,
        warmup_steps = 500,
        logging_steps=1000,
        learning_rate = 2e-4,
        num_train_epochs=5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/32 [00:00<?, ? examples/s]

#### Start training

In [20]:
import os
os.environ["WANDB_DISABLED"] = "true"

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 32 | Num Epochs = 5 | Total steps = 80
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 2 x 1) = 2
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss


TrainOutput(global_step=80, training_loss=3.373370361328125, metrics={'train_runtime': 289.0228, 'train_samples_per_second': 0.554, 'train_steps_per_second': 0.277, 'total_flos': 1161689346908160.0, 'train_loss': 3.373370361328125})

### 4. Test

In [21]:
prompt = "You are an expert Bahnar translator! Below is an instruction that describes a task. Write a response that appropriately completes the request.  ### Instruction: \n Translate the Bahnar input text into English. \n" + ba_en_train_dataset['Bahnar'][94] +"\n ### Response: \n"
print(prompt)

You are an expert Bahnar translator! Below is an instruction that describes a task. Write a response that appropriately completes the request.  ### Instruction: 
 Translate the Bahnar input text into English. 
Sôlômôn hơ-oei kơpal tang-dŏ pơtao Đawit 'bă sư păng dêh char pơtao sư kơjăp 'lơ̆ng tơpă

 ### Response: 



In [22]:
print("result")
display_markdown(prompt_inference(prmpt=prompt),raw=True)

result


 
So, I think that's a good place to start.


In [23]:
ba_en_train_dataset['text'][90]

"You are an expert Bahnar translator! Below is an instruction that describes a task. Write a response that appropriately completes the request.  ### Instruction: \n Translate the Bahnar input text into English. \nLu sư kư̆m pơ̆ng pơlôch 'bar 'nu bơngai tơtông jê̆ hơdai Yêsu, minh 'nu gah 'ma, minh 'nu gah 'ngiĕo kơ Sư\n\n ### Response: \nThen were there two thieves crucified with him, one on the right hand, and another on the left\n<|end_of_text|>"

#### Process the whole test dataset

In [26]:
import torch
# from progress.bar import Bar

def evaluate_model(model, encode_tokenizer, decode_tokenizer, dataset, device):
    model.eval()
    prt = True

    predictions, references = [], []
    for i in range(len(dataset)):
            print(i)
            example = dataset[i]
            prmpt = "You are an expert Bahnar translator! Below is an instruction that describes a task. Write a response that appropriately completes the request.  ### Instruction: \n Translate the Bahnar input text into English. \n" + example['Bahnar'] +"\n ### Response: \n"
            FastLanguageModel.for_inference(model) # Enable native 2x faster inference
            inputs = encode_tokenizer(
            [
                prmpt
            ], return_tensors = "pt").to("cuda")
            
            outputs = model.generate(**inputs, max_new_tokens = 512, temperature=0.1, use_cache = True)
            pred = decode_tokenizer.batch_decode(outputs)[0].split("### Response:")[-1].split("EOS_TOKEN")[0]
            predictions.append([pred])
    
            references.append([example['English']])

    return predictions, references

test_examples = [{'Bahnar': ex['Bahnar'], 'English': ex['English']} for ex in ba_en_test_dataset]
predictions, references = evaluate_model(model, tokenizer, tokenizer, test_examples, "cuda")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

#### Observe the format

In [27]:
predictions[0], references[0]

([' \nAnd then, the next day, we went to the beach.\n<|end_of_text|>'],
 ['I release you, but can do no more.\n'])

In [28]:
preds = [text[0].split(EOS_TOKEN)[0].strip() for text in predictions]
refs = [text[0].strip() for text in references]


#### Calculate metrics

In [29]:
import sacrebleu

bleu = sacrebleu.corpus_bleu(preds, refs)
print("BLEU: ", round(bleu.score, 2))

# Calculate CHRF
chrf = sacrebleu.corpus_chrf(preds, refs)
print("CHRF:", round(chrf.score, 2))

# Calculate TER
metric = sacrebleu.metrics.TER()
ter = metric.corpus_score(preds, refs)
print("TER:", round(ter.score, 2))

BLEU:  0.82
CHRF: 13.59
TER: 979.56


In [30]:
for i in range(10):
    print(f"Prediction {i+1}: {preds[i]}")
    print(f"Reference {i+1}: {refs[i]}")

Prediction 1: And then, the next day, we went to the beach.
Reference 1: I release you, but can do no more.
Prediction 2: And they will be able to do that in the future.
Reference 2: You have as much right to be a knight as any man.
Prediction 3: And I'm not sure.
Reference 3: No. Leave me alone.
Prediction 4: I don't know.
Reference 4: I'm here to help!
Prediction 5: If you don't know, ask.
Reference 5: All you wanna do is pee.
Prediction 6: And now we see the same thing happening in Tunisia and Egypt, where the people are rising up and demanding change.
Reference 6: And these platforms were certainly very helpful to activists in Tunisia and Egypt this past spring and beyond.
Prediction 7: And then Charlie said, "I'm not sure I understand what you're saying."
Reference 7: He's an architect, and Charlie is deeply concerned about global climate change.
Prediction 8: And they will be able to do that.
Reference 8: She took my little girl.
Prediction 9: Oh, I don't think I can do that.
Ref

In [ ]:
model.push_to_hub("tuongdc03/model_name", token = hf_token)
tokenizer.push_to_hub("tuongdc03/model_name", token = hf_token)